# 理论基础

### 架构:
* <img src="../../images/javaee/dubbo-architecture.jpg" width="400px">
* 0, 服务容器负责启动，加载，运行服务提供者。
* 1, 服务提供者在启动时，向注册中心注册自己提供的服务。
* 2, 服务消费者在启动时，向注册中心订阅自己所需的服务。
* 3, 注册中心返回服务提供者地址列表给消费者，如果有变更，注册中心将基于长连接推送变更数据给消费者。
* 4, 服务消费者，从提供者地址列表中，基于软负载均衡算法，选一台提供者进行调用，如果调用失败，再选另一台调用。
* 5, 服务消费者和提供者，在内存中累计调用次数和调用时间，定时每分钟发送一次统计数据到监控中心。

### 框架细节
* 
<table>
    <tr>
        <td><center><h3>框架总图</h3></center></td>
        <td><center><h3>框架总图中的调用链</h3></center></td>
    </tr>
    <tr>
        <td><img src="../../images/javaee/dubbo-framework.jpg" width="500px"></td>
        <td><img src="../../images/javaee/dubbo-extension.jpg" width="450px"></td></td>
    </tr>
    <tr>
        <td><center><h3>服务注册/暴露时序图</h3></center></td>
        <td><center><h3>服务引用/调用时序图</h3></center></td>
    </tr>
    <tr>
        <td><img src="../../images/javaee/dubbo-export.jpg" width="500px"></td></td>
        <td><img src="../../images/javaee/dubbo-refer.jpg" width="450px"></td></td>
    </tr>
</table>

### 框架实现细节
* 参考链接：https://dubbo.incubator.apache.org/zh-cn/docs/dev/implementation.html
* 初始化过程细节
    * 解析服务
        * 基于 dubbo.jar 内的 META-INF/spring.handlers 配置，Spring 在遇到 dubbo 名称空间时，会回调 DubboNamespaceHandler。
        * 所有 dubbo 的标签，都统一用 DubboBeanDefinitionParser 进行解析，基于一对一属性映射，将 XML 标签解析为 Bean 对象。
        * 在ServiceConfig.export()或ReferenceConfig.get()初始化时，将Bean对象转换URL格式，所有Bean属性转成URL的参数。
        * 然后将 URL 传给 协议扩展点，基于扩展点的 扩展点自适应机制，根据 URL 的协议头，进行不同协议的服务暴露或引用。
    * 暴露服务
        * 只暴露服务端口：
            * 在没有注册中心，直接暴露提供者的情况下，ServiceConfig解析出的URL的格式为：dubbo://service-host/com.foo.FooService?version=1.0.0。
            * 基于扩展点自适应机制，通过 URL 的 dubbo:// 协议头识别，直接调用 DubboProtocol的 export() 方法，打开服务端口。
        * 向注册中心暴露服务：
            * 在有注册中心，需要注册提供者地址的情况下，ServiceConfig 解析出的 URL 的格式为: registry://registry-host/com.alibaba.dubbo.registry.RegistryService?export=URL.encode("dubbo://service-host/com.foo.FooService?version=1.0.0")，
            * 基于扩展点自适应机制，通过 URL 的 registry:// 协议头识别，就会调用 RegistryProtocol 的 export() 方法，将 export 参数中的提供者 URL，先注册到注册中心。
            * 再重新传给 Protocol 扩展点进行暴露： dubbo://service-host/com.foo.FooService?version=1.0.0，然后基于扩展点自适应机制，通过提供者 URL 的 dubbo:// 协议头识别，就会调用 DubboProtocol 的 export() 方法，打开服务端口。
        * Dubbo 协议的 Invoker 转为 Exporter 发生在 DubboProtocol 类的 export 方法，它主要是打开 socket 侦听服务
    * 引用服务
        * 直连引用服务：
            * 在没有注册中心，直连提供者的情况下，ReferenceConfig 解析出的 URL 的格式为：dubbo://service-host/com.foo.FooService?version=1.0.0。
            * 基于扩展点自适应机制，通过 URL 的 dubbo:// 协议头识别，直接调用 DubboProtocol 的 refer() 方法，返回提供者引用。
        * 从注册中心发现引用服务：
            * 在有注册中心，通过注册中心发现提供者地址的情况下，ReferenceConfig 解析出的 URL 的格式为： registry://registry-host/com.alibaba.dubbo.registry.RegistryService?refer=URL.encode("consumer://consumer-host/com.foo.FooService?version=1.0.0")。
            * 基于扩展点自适应机制，通过 URL 的 registry:// 协议头识别，就会调用 RegistryProtocol 的 refer() 方法，基于 refer 参数中的条件，查询提供者 URL，如： dubbo://service-host/com.foo.FooService?version=1.0.0。
            * 基于扩展点自适应机制，通过提供者 URL 的 dubbo:// 协议头识别，就会调用 DubboProtocol 的 refer() 方法，得到提供者引用。
            * 然后 RegistryProtocol 将多个提供者引用，通过 Cluster 扩展点，伪装成单个提供者引用返回。

****

# 实践指南
* 配置示例：https://dubbo.incubator.apache.org/zh-cn/docs/user/configuration/xml.html
* 功能示例：https://dubbo.incubator.apache.org/zh-cn/docs/user/demos/preflight-check.html
* 配置属性集合：https://dubbo.incubator.apache.org/zh-cn/docs/user/references/xml/introduction.html
* 协议汇总：https://dubbo.incubator.apache.org/zh-cn/docs/user/references/protocol/introduction.html
* 注册中心：https://dubbo.incubator.apache.org/zh-cn/docs/user/references/registry/introduction.html
* 最佳实践：https://dubbo.incubator.apache.org/zh-cn/docs/user/recommend.html

****

# RPC or Remote Procedure call vs RMI or Remote Method Invokation
* 详细课件介绍: https://cseweb.ucsd.edu/classes/sp16/cse291-e/applications/ln/lecture3.html
* <b>Definition of RPC</b>
    * <i>Remote Procedure Call (RPC) is a programming language feature devised for the distributed computing and based on semantics of local procedure calls. It is the most common forms of remote service and was designed as a way to abstract the procedure call mechanism to use between systems connected through a network. It is similar to IPC mechanism where the operating system allows the processes to manage shared data and deal with an environment where different processes are executing on separate systems and necessarily require message-based communication.</i>
    * RPC is implemented through the given steps: 
        * 
        <table>
            <tr>
                <td><img src="../../images/javaee/RPC-mechanism.jpg" width="400px"></td>
                <td><img src="../../images/javaee/rpcoverview.jpg" width="400px"></td>
            </tr>
        </table>
        * The client process calls the client stub with parameters, and its execution is suspended until the call is completed.
        * The parameters are then translated into machine-independent form by marshalling through client stub. Then the message is prepared which contain the representation of the parameters.
        * To find the identity of the site the client stub intercommunicate with name server at which remote procedure exists.
        * Using blocking protocol the client stub sends the message to the site where remote procedure call exists. This step halt the client stub until it gets a reply.
        * The server site receives the message sent from the client side and converts it into machine specific format.
        * Now server stub executes a call on the server procedure along with the parameters, and the server stub is discontinued till the procedure gets completed.
        * The server procedure returns the generated results to the server stub, and the results get converted into machine-independent format at server stub and create a message containing the results.
        * The result message is sent to the client stub which is converted back into machine specific format suitable for the client stub.
        * At last client, stub returns the results to the client process.
* <b>Definition of RMI</b>
    * <i>Remote Method Invocation (RMI) is similar to RPC but is language specific and a feature of java. A thread is permitted to call the method on a remote object. To maintain the transparency on the client and server side, it implements remote object using stubs and skeletons. The stub resides with the client and for the remote object it behaves as a proxy.</i>
    * RMI is implemented through the given steps: 
        * 
        <table>
            <tr>
                <td><img src="../../images/javaee/RMI-mechanism.jpg" width="400px"></td>
                <td><img src="../../images/javaee/rmi.gif" width="400px"></td>
            </tr>
        </table>
        * When a client calls a remote method, the stub for the remote method is called. 
        * The client stub is accountable for creating and sending the parcel containing the name of a method and the marshalled parameters, and the skeleton is responsible for receiving the parcel.
        * The skeleton unmarshals parameters and invokes the desired method on the server. 
        * The skeleton marshals the given value (or exceptions) with the parcel and sends it to client stub. 
        * The stub reassembles the return parcel and sends it to the client.
* <b>Key Differences Between RPC and RMI</b>
    * RPC supports procedural programming paradigms thus is C based, while RMI supports object-oriented programming paradigms and is java based.
    * The parameters passed to remote procedures in RPC are the ordinary data structures. On the contrary, RMI transits objects as a parameter to the remote method.
    * RPC can be considered as the older version of RMI, and it is used in the programming languages that support procedural programming, and it can only use pass by value method. As against, RMI facility is devised based on modern programming approach, which could use pass by value or reference. Another advantage of RMI is that the parameters passed by reference can be changed.
    * RPC protocol generates more overheads than RMI.
    * The parameters passed in RPC must be “in-out” which means that the value passed to the procedure and the output value must have the same datatypes. In contrast, there is no compulsion of passing “in-out” parameters in RMI.
    * In RPC, references could not be probable because the two processes have the distinct address space, but it is possible in case of RMI.